## Messing around with some new classifiers

## Import Packages

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk

from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, SVC
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression
# also look into SGDClassifier

## Loading file

In [2]:
df = pd.read_json('data/Sports_and_Outdoors_Reviews_training.json', lines=True)

In [3]:
def text_process(text):
    text = ' '.join(str(text).split())
    return text

## group by product and aggregate all the review summary text

In [4]:
grouped_df = df.groupby("asin")
grouped_lists = grouped_df[['summary', 'reviewText']].apply(text_process).reset_index()
grouped_lists

<ipython-input-4-72a4c2da08ef>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped_lists = grouped_df["summary", 'reviewText'].apply(text_process).reset_index()


,asin,0
0,00018C9635D55E22BF157AA13E91226F,summary \ 2045677 Five Stars 2045678 I recomme...
1,0001DE3A462B5C5D33AF3BC1053FC20C,"summary \ 2142792 Very portable, fairly durabl..."
2,00022ACC61318C98DA944B9BABD9E5AB,"summary \ 434812 Great product, poor shipping...."
3,0002C8404EBEDA230E4B66A85CEC5503,summary \ 417817 Small and Cheap 417818 One St...
4,00034EBDF69991833D05B51EE7B11234,summary \ 91838 Pretty Good 91839 Escrima stic...
...,...,...
83744,FFFCB2EBE4D59DA2E256396B12F59FB1,"summary \ 1262012 For The Price, Resin Pads Al..."
83745,FFFEE00C6052E1A688F4639D650AA50D,summary \ 593487 Good purchase 593488 Great ba...
83746,FFFEE7703FE466554E6B5F9C21F09297,summary \ 593764 Cheapo zipper not for me 5937...
83747,FFFF67EAA043C2DB092DBC8934077556,summary \ 1496788 No specs -- no deal 1496789 ...


## get the mean of all ratings for a given product

In [5]:
mean_df = grouped_df['overall'].mean()
mean_df = mean_df.reset_index()
mean_df

,asin,overall
0,00018C9635D55E22BF157AA13E91226F,4.090909
1,0001DE3A462B5C5D33AF3BC1053FC20C,3.909091
2,00022ACC61318C98DA944B9BABD9E5AB,4.698413
3,0002C8404EBEDA230E4B66A85CEC5503,3.400000
4,00034EBDF69991833D05B51EE7B11234,4.214286
...,...,...
83744,FFFCB2EBE4D59DA2E256396B12F59FB1,4.941176
83745,FFFEE00C6052E1A688F4639D650AA50D,4.452381
83746,FFFEE7703FE466554E6B5F9C21F09297,4.310345
83747,FFFF67EAA043C2DB092DBC8934077556,4.600000


## merge product mean ratings + aggregated review summary text

In [6]:
final_df = pd.merge(grouped_lists, mean_df, on="asin")
final_df

,asin,0,overall
0,00018C9635D55E22BF157AA13E91226F,summary \ 2045677 Five Stars 2045678 I recomme...,4.090909
1,0001DE3A462B5C5D33AF3BC1053FC20C,"summary \ 2142792 Very portable, fairly durabl...",3.909091
2,00022ACC61318C98DA944B9BABD9E5AB,"summary \ 434812 Great product, poor shipping....",4.698413
3,0002C8404EBEDA230E4B66A85CEC5503,summary \ 417817 Small and Cheap 417818 One St...,3.400000
4,00034EBDF69991833D05B51EE7B11234,summary \ 91838 Pretty Good 91839 Escrima stic...,4.214286
...,...,...,...
83744,FFFCB2EBE4D59DA2E256396B12F59FB1,"summary \ 1262012 For The Price, Resin Pads Al...",4.941176
83745,FFFEE00C6052E1A688F4639D650AA50D,summary \ 593487 Good purchase 593488 Great ba...,4.452381
83746,FFFEE7703FE466554E6B5F9C21F09297,summary \ 593764 Cheapo zipper not for me 5937...,4.310345
83747,FFFF67EAA043C2DB092DBC8934077556,summary \ 1496788 No specs -- no deal 1496789 ...,4.600000


## classify as awesome/not

In [7]:
# function to use for lambda to categorize as awesome, not awesome
def df_iter(overall):
    if overall > 4.5:
        result = 'awesome'
    else:
        result = 'not'
        
    return result

In [8]:
final_df['class'] = final_df.apply(lambda row: df_iter(row['overall']), axis=1)
final_df

,asin,0,overall,class
0,00018C9635D55E22BF157AA13E91226F,summary \ 2045677 Five Stars 2045678 I recomme...,4.090909,not
1,0001DE3A462B5C5D33AF3BC1053FC20C,"summary \ 2142792 Very portable, fairly durabl...",3.909091,not
2,00022ACC61318C98DA944B9BABD9E5AB,"summary \ 434812 Great product, poor shipping....",4.698413,awesome
3,0002C8404EBEDA230E4B66A85CEC5503,summary \ 417817 Small and Cheap 417818 One St...,3.400000,not
4,00034EBDF69991833D05B51EE7B11234,summary \ 91838 Pretty Good 91839 Escrima stic...,4.214286,not
...,...,...,...,...
83744,FFFCB2EBE4D59DA2E256396B12F59FB1,"summary \ 1262012 For The Price, Resin Pads Al...",4.941176,awesome
83745,FFFEE00C6052E1A688F4639D650AA50D,summary \ 593487 Good purchase 593488 Great ba...,4.452381,not
83746,FFFEE7703FE466554E6B5F9C21F09297,summary \ 593764 Cheapo zipper not for me 5937...,4.310345,not
83747,FFFF67EAA043C2DB092DBC8934077556,summary \ 1496788 No specs -- no deal 1496789 ...,4.600000,awesome


In [9]:
# reasonably balanced!
final_df['class'].value_counts()

not        46668
awesome    37081
Name: class, dtype: int64

## Prep + vectorize text, feed into SVC classifier

In [28]:
stemmer = SnowballStemmer("english", ignore_stopwords=True)
stop_words = stopwords.words("english")
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(preprocessor=stemmer.stem, stop_words=stop_words, ngram_range = (1,2), tokenizer=token.tokenize)
text_counts = cv.fit_transform(final_df[0])

In [ ]:
# PICK N CHOOSE BETWEEN TFIDF AND COUNT
tv = TfidfVectorizer(preprocessor=stemmer.stem, stop_words=stop_words, ngram_range = (1,3), tokenizer=token.tokenize)
text_counts = tv.fit_transform(final_df[0])

In [11]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(text_counts, final_df['class'], test_size = 0.25, random_state = 5)

This next one is horrible don't do it

In [14]:
#Basic SVC (much slower but more options)
SVC_classifier = SVC(kernel='poly', degree=3)#,class_weight='balanced')
SVC_classifier.fit(X_train, y_train)
y_score = SVC_classifier.decision_function(X_test)
y_pred_SVC = SVC_classifier.predict(X_test)

In [11]:
#Strictly Linear SVC (v fast but not many options)
SVC_classifier = LinearSVC()
SVC_classifier.fit(X_train, y_train)
y_score = SVC_classifier.decision_function(X_test)
y_pred_SVC = SVC_classifier.predict(X_test)

f1_score_SVC = metrics.f1_score(y_test, y_pred_SVC, average='weighted')
auc_score_SVC = metrics.roc_auc_score(y_test, y_score, average='weighted', labels=["not","awesome"])
precision_score_SVC = metrics.average_precision_score(y_test, y_score, average='weighted', pos_label="not")

print('SVC F1: ' + str('{:04.2f}'.format(f1_score_SVC*100)) + '%')
print('SVC Precision-Recall: ' + str('{:04.2f}'.format(precision_score_SVC*100)) + '%')
print('SVC AUC: ' + str('{:04.2f}'.format(auc_score_SVC*100)) + '%')

#### NEW STUFF

In [54]:
#AdaBoost with linear SVC
boost_classifier = AdaBoostClassifier(LinearSVC(loss='squared_hinge'),
                                      algorithm='SAMME',
                                      learning_rate=0.125,
                                      n_estimators=200)
boost_classifier.fit(X_train, y_train)
y_score = boost_classifier.decision_function(X_test)
y_pred_boost = boost_classifier.predict(X_test)

In [55]:
# checking AdaBoost accuracy
f1_score_boost = metrics.f1_score(y_test, y_pred_boost, average='weighted')
precision_score_boost = metrics.average_precision_score(y_test, y_score, average='weighted', pos_label="not")

print('Boosted SVC F1: ' + str('{:04.2f}'.format(f1_score_boost*100)) + '%')
print('Boosted SVC Precision-Recall: ' + str('{:04.2f}'.format(precision_score_boost*100)) + '%')

Boosted SVC F1: 78.40%
Boosted SVC Precision-Recall: 89.80%


In [52]:
#Bagging with linear SVC
bag_classifier = BaggingClassifier(base_estimator=LinearSVC(loss='hinge'),
                                   max_samples=0.4,
                                   n_estimators=100)
bag_classifier.fit(X_train, y_train)
y_score = bag_classifier.decision_function(X_test)
y_pred_bag = bag_classifier.predict(X_test)

In [53]:
# checking bagging accuracy
f1_score_bag = metrics.f1_score(y_test, y_pred_bag, average='weighted')
precision_score_bag = metrics.average_precision_score(y_test, y_score, average='weighted', pos_label="not")

print('Boosted SVC F1: ' + str('{:04.2f}'.format(f1_score_bag*100)) + '%')
print('Boosted SVC Precision-Recall: ' + str('{:04.2f}'.format(precision_score_bag*100)) + '%')

Boosted SVC F1: 78.28%
Boosted SVC Precision-Recall: 89.36%


In [ ]:
#Bagging with logistic regression
# bag_classifier = BaggingClassifier(base_estimator=LogisticRegression(penalty='l2', solver='sag', max_iter=10000),
#                                    max_samples=0.4,
#                                    n_estimators=100)
# bag_classifier.fit(X_train, y_train)
# y_score = bag_classifier.decision_function(X_test)
# y_pred_bag = bag_classifier.predict(X_test)

In [ ]:
# checking accuracy
# f1_score_bag = metrics.f1_score(y_test, y_pred_bag, average='weighted')
# precision_score_bag = metrics.average_precision_score(y_test, y_score, average='weighted', pos_label="not")

# print('Bagged LR F1: ' + str('{:04.2f}'.format(f1_score_bag*100)) + '%')
# print('Bagged LR Precision-Recall: ' + str('{:04.2f}'.format(precision_score_bag*100)) + '%')

## k-means cross validation (I didn't touch anything after this)

In [30]:
# splitting data for testing and training to see how well model performs

SVC_classifier = LinearSVC()
from sklearn.model_selection import StratifiedShuffleSplit
from statistics import mean
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=5)
# X_train, X_test, Y_train, Y_test = train_test_split(text_counts, df_sorted['overall'], test_size = 0.25, random_state = 5)
count = 1
X = text_counts
y = final_df['class']
f1_scores, auc_scores = list(), list()
for train_idx, test_idx in sss.split(X, y):
    print ("Group" , count)
    count += 1
    X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx]
    SVC_classifier.fit(X_train, y_train)
    y_score = SVC_classifier.decision_function(X_test)
    y_pred_SVC = SVC_classifier.predict(X_test)
    # checking accuracy
    f1_score_SVC = metrics.f1_score(y_test, y_pred_SVC, average='weighted')
    precision_score_SVC = metrics.average_precision_score(y_test, y_score, average='weighted', pos_label="not")
    
    f1_scores.append(f1_score_SVC)
    auc_scores.append(precision_score_SVC)
    
    print('SVC F1: ' + str('{:04.2f}'.format(f1_score_SVC*100)) + '%')
    print('SVC Precision-Recall AUC: ' + str('{:04.2f}'.format(precision_score_SVC*100)) + '%')
    print()
print('F1 average: ' + str('{:04.2f}'.format(mean(f1_scores)*100)) + '%')
print('Precision-Recall AUC average: ' + str('{:04.2f}'.format(mean(auc_scores)*100)) + '%')



Group 1
SVC F1: 78.44%
SVC Precision-Recall AUC: 89.43%

Group 2
SVC F1: 78.87%
SVC Precision-Recall AUC: 89.82%

Group 3
SVC F1: 77.94%
SVC Precision-Recall AUC: 89.11%

Group 4
SVC F1: 78.87%
SVC Precision-Recall AUC: 89.83%

Group 5
SVC F1: 78.65%
SVC Precision-Recall AUC: 89.63%

F1 average: 78.55%
Precision-Recall AUC average: 89.57%


# Predictions on Test Dataset

In [40]:
# Read in raw data
test_df = pd.read_json('data/Sports_and_Outdoors_Reviews_test.json', lines=True)

group_df = test_df.groupby("asin")
group_lists = group_df["summary"].apply(text_process).reset_index()

test_counts = cv.transform(group_lists['summary'])

In [41]:
predictions = SVC_classifier.predict(test_counts)
group_lists['predictions'] = predictions
group_lists

,asin,summary,predictions
0,00001378E0675643F36F8B5147FDA6D0,158023 Awesome Product 158024 very nice rifle ...,awesome
1,00010FEA8BF06921C276FE4DB4B63AB2,151462 actually fits the Governor and makes ca...,awesome
2,0001DCCBFABAC9E1073A1F87A393E5BA,464277 One of the best options out there for m...,awesome
3,0003494756240B4FC0579229A91398F7,284314 Fun at the range. 284315 Great fun 2843...,not
4,00064AD7C050018B509A034445C74890,177600 Tackiest grip ever 177601 Sure Hands 17...,awesome
...,...,...,...
20933,FFE4079FBEC0377CA942C4C3E3FDB127,330833 Kid's snorkel vest. 330834 Couldn't blo...,not
20934,FFE64F533026658E64520B5F50C3338D,344744 I don't know that my 4 yr old nephew wo...,not
20935,FFE8D7BE871372F0FBD09DB5C962E335,58975 Five Stars 58976 Two Stars 58977 Four St...,not
20936,FFEBF00219AA7DC96F0D3584EF0581CD,525044 Fun Fun Fun ... 525045 5 Star Value - N...,not


In [42]:
output = group_lists[['asin','predictions']].sort_values('asin')
output

,asin,predictions
0,00001378E0675643F36F8B5147FDA6D0,awesome
1,00010FEA8BF06921C276FE4DB4B63AB2,awesome
2,0001DCCBFABAC9E1073A1F87A393E5BA,awesome
3,0003494756240B4FC0579229A91398F7,not
4,00064AD7C050018B509A034445C74890,awesome
...,...,...
20933,FFE4079FBEC0377CA942C4C3E3FDB127,not
20934,FFE64F533026658E64520B5F50C3338D,not
20935,FFE8D7BE871372F0FBD09DB5C962E335,not
20936,FFEBF00219AA7DC96F0D3584EF0581CD,not


In [43]:
output.to_csv('predictions.csv', index=False)